# Analyzing D&D Datasets

## Spells

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import lxml

In [2]:
with open('3.5 Spells.xml') as file:
    spells = file.read()
# len(spells)


In [3]:
# for some reason this wouldn't parse as xml, but it will work with html
soup = BeautifulSoup(spells, 'html')


In [4]:
desc = soup.spelldesc

In [5]:
lists = soup.spelllists

In [6]:
len(list(lists.children))

17

In [67]:
list(lists.contents)[1].name

'bard'

In [ ]:
list(lists.contents)[1].get_text()

In [ ]:
list(lists.contents)[1].get_text().split('\n')

In [60]:
len(list(lists.descendants))

20911

In [7]:
def is_new_line(item):
    if item == '\n':
        return True
    return False


In [37]:
def create_spell_dict():
    spell_desc_list = []
    children = list(desc.children)
    # the DnD spell (or a new line - ignore those)
    for spell_tag in children:

        if is_new_line(spell_tag.string):
            continue
            
        spell_name = spell_tag.name # can be used for the dictionary
        
#         print(spell_name)
        descriptions = list(spell_tag.children)
        
        descript_dict = {}
        for descript in descriptions:
            if is_new_line(descript):
                continue

            descript_dict[descript.name] = descript.string
            
        spell_desc_list.append(descript_dict)
#         break

    return spell_desc_list


In [39]:
spell_desc_list = create_spell_dict()
# print(spell_desc_list)

In [40]:
df = pd.DataFrame.from_dict(spell_desc_list)
df

,name,school,level,components,castingtime,range,effect,duration,save,sr,description,shortdescription
0,Acid Arrow,Conjuration (Creation) [Acid],Sor/Wiz 2,"V, S, M, F",1 standard action,Long (400 ft. + 40 ft./level),One arrow of acid,1 round + 1 round per three levels,None,No,None,Ranged touch attack; 2d4 damage for 1 round +1...
1,Acid Fog,Conjuration (Creation) [Acid],"Sor/Wiz 6, Water 7","V, S, M/DF",1 standard action,Medium (100 ft. + 10 ft./level),"Fog spreads in 20-ft. radius, 20 ft. high",1 round/level,None,No,None,Fog deals acid damage.
2,Acid Splash,Conjuration (Creation) [Acid],Sor/Wiz 0,"V, S",1 standard action,Close (25 ft. + 5 ft./2 levels),One missile of acid,Instantaneous,None,No,You fire a small orb of acid at the target. Yo...,Orb deals 1d3 acid damage.
3,Aid,Enchantment (Compulsion) [Mind-Affecting],"Clr 2, Good 2, Luck 2","V, S, DF",1 standard action,Touch,Living creature touched,1 min./level,None,Yes (harmless),None,"+1 on attack rolls, +1 against fear, 1d8 tempo..."
4,Air Walk,Transmutation [Air],"Air 4, Clr 4, Drd 4","V, S, DF",1 standard action,Touch,Creature (Gargantuan or smaller) touched,10 min./level,None,Yes (harmless),None,Subject treads on air as if solid (climb at 45...
...,...,...,...,...,...,...,...,...,...,...,...,...
601,Wood Shape,Transmutation,Drd 2,"V, S, DF",1 standard action,Touch,One touched piece of wood no larger than 10 cu...,Instantaneous,Will negates (object),Yes (object),None,Rearranges wooden objects to suit you.
602,Word of Chaos,"Evocation [Chaotic, Sonic]","Chaos 7, Clr 7",V,1 standard action,40 ft.,Nonchaotic creatures in a 40-ft.- radius sprea...,Instantaneous,None or Will negates; see text,Yes,None,"Kills, confuses, stuns, or deafens nonchaotic ..."
603,Word of Recall,Conjuration (Teleportation),"Clr 6, Drd 8",V,1 standard action,Unlimited,You and touched objects or other willing creat...,Instantaneous,"None or Will negates (harmless, object)","No or Yes (harmless, object)",None,Teleports you back to designated place.
604,Zone of Silence,Illusion (Glamer),Brd 4,"V, S",1 round,Personal,5-ft.-radius emanation centered on you,1 hour/level (D),NaN,NaN,None,Keeps eavesdroppers from overhearing conversat...


In [8]:

def create_class_spell_dict():
    class_spells = {}
    children = list(lists.children)
    # the DnD class (or a new line - ignore those)
    for class_tag in children:

        if is_new_line(class_tag.string):
            continue
        
        class_name = class_tag.name # can be used for the dictionary
        # this the groups where each child is the different spell levels
        class_groups = class_tag.find('groups')
        if not class_groups:
            continue
        
        class_spell_lvls = list(class_groups.children)

        spell_lvls = {}
        # the spell levels (or a new line - ignore those)
        for spell_lvl in class_spell_lvls:
            if is_new_line(spell_lvl):
                continue

            # spell_lvl.name can be used in the dict
            spells_for_CSL = spell_lvl.find('spells')

            spells_for_lvl = [] # making a list of spells for each spell level
            for spell in spells_for_CSL:
                if is_new_line(spell):
                    continue

                spells_for_lvl.append(spell.name)
                # print(class_tag.name, spell_lvl.name, spell.name)

            spell_lvls[spell_lvl.name] = spells_for_lvl

        class_spells[class_name] = spell_lvls
        
    return class_spells

In [27]:
dict_class_spells = create_class_spell_dict()

In [28]:
dict_class_spells

{'bard': {'level0': ['dancinglights',
   'daze',
   'detectmagic',
   'flare',
   'ghostsound',
   'knowdirection',
   'light',
   'lullaby',
   'magehand',
   'mending',
   'message',
   'openclose',
   'prestidigitation',
   'readmagic',
   'resistance',
   'summoninstrument'],
  'level1': ['alarm',
   'animaterope',
   'causefear',
   'charmperson',
   'comprehendlanguages',
   'confusionlesser',
   'curelightwounds',
   'detectsecretdoors',
   'disguiseself',
   'erase',
   'expeditiousretreat',
   'featherfall',
   'grease',
   'hideouslaughter',
   'hypnotism',
   'identify',
   'magicaura',
   'magicmouth',
   'obscureobject',
   'removefear',
   'silentimage',
   'sleep',
   'summonmonsteri',
   'undetectablealignment',
   'unseenservant',
   'ventriloquism'],
  'level2': ['alterself',
   'animalmessenger',
   'animaltrance',
   'blindnessdeafness',
   'blur',
   'calmemotions',
   'catsgrace',
   'curemoderatewounds',
   'darkness',
   'dazemonster',
   'delaypoison',
   'dete

In [ ]:
# will need to modify to move wizard schools & cleric domains to another dict

In [ ]:
# desc.find_next_sibling()

In [29]:
# soup.find_parents()

[]

In [30]:
# soup.find_next_siblings()

[]

In [35]:
# len(soup)

1

In [ ]:
# lists.prettify()

In [200]:
df_spells = pd.DataFrame.from_dict(dict_class_spells)

In [202]:
df_spells.head(60)

,bard,cleric,druid,paladin,ranger,wizard,wizardschools
level0,"[dancinglights, daze, detectmagic, flare, ghos...","[createwater, cureminorwounds, detectmagic, de...","[createwater, cureminorwounds, detectmagic, de...",NaN,NaN,"[acidsplash, arcanemark, dancinglights, daze, ...",[resistance]
level1,"[alarm, animaterope, causefear, charmperson, c...","[bane, bless, blesswater, causefear, command, ...","[calmanimals, charmanimal, curelightwounds, de...","[bless, blesswater, blessweapon, createwater, ...","[alarm, animalmessenger, calmanimals, charmani...","[alarm, animaterope, burninghands, causefear, ...","[alarm, endureelements, holdportal, protection..."
level2,"[alterself, animalmessenger, animaltrance, bli...","[aid, alignweapon, augury, bearsendurance, bul...","[animalmessenger, animaltrance, barkskin, bear...","[bullsstrength, delaypoison, eaglessplendor, o...","[barkskin, bearsendurance, catsgrace, cureligh...","[acidarrow, alterself, arcanelock, bearsendura...","[arcanelock, obscureobject, protectionfromarro..."
level3,"[blink, charmmonster, clairaudienceclairvoyanc...","[animatedead, bestowcurse, blindnessdeafness, ...","[calllightning, contagion, curemoderatewounds,...","[curemoderatewounds, daylight, discernlies, di...","[commandplants, curemoderatewounds, darkvision...","[arcanesight, blink, clairaudienceclairvoyance...","[dispelmagic, explosiverunes, magiccircleagain..."
level4,"[breakenchantment, curecriticalwounds, detects...","[airwalk, controlwater, curecriticalwounds, de...","[airwalk, antiplantshell, blight, commandplant...","[breakenchantment, cureseriouswounds, deathwar...","[animalgrowth, communewithnature, cureseriousw...","[animatedead, arcaneeye, bestowcurse, blackten...","[dimensionalanchor, firetrap, globeofinvulnera..."
level5,"[curelightwoundsmass, dispelmagicgreater, drea...","[atonement, breakenchantment, commandgreater, ...","[animalgrowth, atonement, awaken, balefulpolym...",NaN,NaN,"[animalgrowth, balefulpolymorph, blight, break...","[breakenchantment, dismissal, magesprivatesanc..."
level6,"[analyzedweomer, animateobjects, catsgracemass...","[animateobjects, antilifeshell, banishment, be...","[antilifeshell, bearsendurancemass, bullsstren...",NaN,NaN,"[acidfog, analyzedweomer, antimagicfield, bear...","[antimagicfield, dispelmagicgreater, globeofin..."
level7,NaN,"[blasphemy, controlweather, cureseriouswoundsm...","[animateplants, changestaff, controlweather, c...",NaN,NaN,"[arcanesightgreater, banishment, controlundead...","[banishment, sequester, spellturning]"
level8,NaN,"[antimagicfield, cloakofchaos, creategreaterun...","[animalshapes, controlplants, cureseriouswound...",NaN,NaN,"[antipathy, binding, charmmonstermass, clenche...","[dimensionallock, mindblank, prismaticwall, pr..."
level9,NaN,"[astralprojection, energydrain, etherealness, ...","[antipathy, curecriticalwoundsmass, elementals...",NaN,NaN,"[astralprojection, crushinghand, dominatemonst...","[freedom, imprisonment, magesdisjunction, pris..."


In [203]:
df_spells.columns

Index(['bard', 'cleric', 'druid', 'paladin', 'ranger', 'wizard',
       'wizardschools'],
      dtype='object')